# Summarize Online Results

This notebook is used to generate tables for the paper on OMMA algorithm.

## Main results

In [ ]:
#seeds = [1, 13, 23, 2024, 7700]
seeds = [1, 13, 23, 2024, 7700]
results_dir = "../results_online6"
datasets = [
        ("youtube_deepwalk_plt", "\\datasettable{YouTube} ($m = 46, n = 7926$)"), 
        ("eurlex_lexglue_plt", "\\datasettable{Eurlex-LexGlue} ($m = 100, n = 5000$)"), 
        ("mediamill_plt", "\\datasettable{Mediamill} ($m = 101, n = 12914$)"), 
        ("flicker_deepwalk_plt", "\\datasettable{Flickr} ($m = 195, n = 24154$)"), 
        #("rcv1x_plt", "\\datasettable{RCV1X} ($m = 2456, n = 155962$)"), 
        #("amazoncat_plt", "\\datasettable{AmazonCat} ($m = 13330, n = 306784$)"), 
        
        #("eurlex_plt", "Eurlex"), 
        #("eurlex_lightxml", "Eurlex"), 
        #("eurlex_plt", "Eurlex ($m = 3993, n = 3809$)"), 
        #("eurlex_lightxml", "Eurlex ($m = 5600, n = 5000$)"), 
        #("amazoncat_lightxml", "AmazonCat"), 
        #("amazon_lightxml", "Amazon")
        # "wiki500_lightxml", "Wikipedia",
]
metrics = [
    ("micro_f1_k=0", "miF@0", "Micro F1"), 
    ("macro_f1_k=0", "mF@0", "Macro F1"),
    ("macro_f1_k=3", "mF@3", "Macro F1$@3$"),
    #("macro_f1_k=5", f"mF@5", "Macro F1$@5$"),
    ("macro_recall_k=3", "mR@3", "Macro Recall$@3$"), 
    ("macro_precision_k=3", "mP@3", "Macro Prec.$@3$"), 
    #("macro_recall_k=5", f"mR@5", "Macro Recall$@5$"),
    ("macro_gmean_k=0", "mG@0", "Macro G-Mean"),
    ("macro_hmean_k=0", "mH@0", "Macro H-Mean"),
    #("macro_min_tp_tn_k=0", "min_tp_tn@0", "Macro $\\min(\\text{tp},\\text{tn})$"),
    #("macro_precision", f"mP@{k}", "Macro Precision"), 
    #("macro_min_tp_tn", f"min_tp_tn@{k}", "Macro-Min(TP,TN)"),
]

methods = [
    #("default_prediction", "Top-K / $\hat \eta(x) > 0.5$"),
    ("online_default", "Top-$k$ / $\hat \eta \!>\!0.5$"),

    ("ofo", "OFO"),
    ("online_greedy", "Greedy"),
    ("online_frank_wolfe_exp=1.1", "Online-FW"),
    ("omma", "OMMA"),
    #("online_my", "OMMA"),
    ("frank_wolfe", "FW"),
    ("frank_wolfe_on_test", "FW-t"),

    #("ofo_etu", "OFO$(\\hat \\eta)$"),
    #("online_greedy_etu", "Greedy$(\\hat \\eta)$"),
    ("online_frank_wolfe_etu_exp=1.1", "Online-FW$(\\hat \\eta)$"),
    ("omma_etu", "OMMA$(\\hat \\eta)$"),
    #("online_my_etu", "OMMA$(\\hat \\eta)$"),
    ("frank_wolfe_on_train_etu", "FW$(\\hat \\eta)$"),
    ("frank_wolfe_etu", "FW-t$(\\hat \\eta)$"),
    ("frank_wolfe", "FW"),

]

import json
import os
import numpy as np


def load_json(filepath):
    with open(filepath) as file:
        return json.load(file)


def check_cell(c, command):
    if c.strip() == "":
        return ""
    else:
        return f"\\{command}{{{c.strip()}}}"

def wrap_calls(text, command):
    text = text.split("&")
    return "&".join([check_cell(c, command) for c in text])


def wrap_calls_md(text, command):
    text = text.split("|")
    return "|".join([f"{command}{c}{command}" for c in text])


def get_result(dataset, method, metric, metric_key, seeds=seeds):
    results = []
    for s in seeds:
        path1 = f"{results_dir}/{dataset}/{method}_{metric}_s={s}_results.json"
        if os.path.exists(path1):
            method_results = load_json(path1)
        else:
            path2 = f"{results_dir}/{dataset}/{method}_{metric.split('_')[-1]}_s={s}_results.json"
            if os.path.exists(path2):
                method_results = load_json(path2)
            else:
                #print(f"File {path1} or {path2} does not exist")
                continue

        try:
            if metric_key == "pred_utility_history":
                results.append(method_results["pred_utility_history"][-1][-1] * 100)
            else:
                results.append(method_results[metric_key])
        except Exception as e:
            print(e)
            continue
    if len(results):
        results = np.array(results)
        return results.mean(), results.std()
    else:
        return 0, 0

# Create data
def create_main_table(datasets, methods, metrics, seeds, header, rules, format="tex"):
    table = []
    for i, (dataset, dataset_name) in enumerate(datasets):
        if i % 2 == 0 or format != "tex":
            for method, method_name in methods:
                table.append([method_name])
        
        for metric, metric_key, metric_name in metrics:
            table_col = []
            for method, method_name in methods:
                table_col.append(get_result(dataset, method, metric, "pred_utility_history", seeds=seeds)[0])
            
            table_col = np.array(table_col).round(2)
            
            if format == "tex":
                table_col_str = [f"{x:.2f}" if x != 0 else "$\\times$" for x in table_col]
                max_row = np.argsort(table_col)
                max_i = -1
                table_col_str[max_row[max_i]] = wrap_calls(table_col_str[max_row[max_i]], "textbf")
                max_i -= 1
                while table_col[max_row[max_i]] == table_col[max_row[max_i + 1]] and max_i > -len(table_col):
                    table_col_str[max_row[max_i]] = wrap_calls(table_col_str[max_row[max_i]], "textbf")
                    max_i -= 1

                if max_i > -len(table_col):
                    table_col_str[max_row[max_i]] = wrap_calls(table_col_str[max_row[max_i]], "textit")
                    max_i -= 1
                while table_col[max_row[max_i]] == table_col[max_row[max_i + 1]] and max_i > -len(table_col):
                    table_col_str[max_row[max_i]] = wrap_calls(table_col_str[max_row[max_i]], "textit")
                    max_i -= 1
                #print(table_col_str)
            else:
                table_col_str = [f"{x:.2f}" if x != 0 else "x" for x in table_col]
                max_row = np.argsort(table_col)
                max_i = -1
                table_col_str[max_row[max_i]] = wrap_calls_md(table_col_str[max_row[max_i]], "**")
                max_i -= 1
                while table_col[max_row[max_i]] == table_col[max_row[max_i + 1]] and max_i > -len(table_col):
                    table_col_str[max_row[max_i]] = wrap_calls_md(table_col_str[max_row[max_i]], "**")
                    max_i -= 1

                if max_i > -len(table_col):
                    table_col_str[max_row[max_i]] = wrap_calls_md(table_col_str[max_row[max_i]], "*")
                    max_i -= 1
                while table_col[max_row[max_i]] == table_col[max_row[max_i + 1]] and max_i > -len(table_col):
                    table_col_str[max_row[max_i]] = wrap_calls_md(table_col_str[max_row[max_i]], "*")
                    max_i -= 1
            
            m = len(table_col)
            for j, c in enumerate(table_col_str):
                table[-(m - j)].append(c)

    if format == "tex":
        print(header)
    d = 0
    for i, table_row in enumerate(table):        
        if format == "tex":
            if i % len(methods) == 0:
                sec_header = f"\\midrule\n& \\multicolumn{{7}}{{c|}}{{{datasets[d][1]}}} & \\multicolumn{{7}}{{c}}{{{datasets[d + 1][1]}}} \\\\ \n \\midrule"
                d += 2
                print(sec_header)
            
            row = "    " + " & ".join(table_row) + " \\\\"
            for r in rules:
                if i % len(methods) == r:
                    row += "\n    \\midrule"
        else:
            if i % len(methods) == 0:
                print(f"**{datasets[d][1]}**")
                d += 1
                print(header)
            row = "| " + " | ".join(table_row) + " |"
        print(row)

    if format == "tex":
        print("\\bottomrule\n\\end{tabular}")


header1 = "\\begin{tabular}{l|c|cccccc|c|cccccc}"
header2 = "\\toprule\n    Method & Micro & \\multicolumn{6}{c|}{Macro} & Micro & \\multicolumn{6}{c}{Macro} \\\\"
header3 = "    & " + " & ".join([f"{metric[2].split(' ')[-1]}" for metric in metrics]) + " & " + " & ".join([f"{metric[2].split(' ')[-1]}" for metric in metrics]) + " \\\\"
header = f"{header1}\n{header2}\n{header3}"

header_md = """
| Method | Mi-F1 | Ma-F1 | Ma-F1@3 | Ma-R@3 | Ma-P@3 | Ma-G-Mean | Ma-H-Mean |
|:-------|------:|------:|--------:|-------:|-------:|----------:|----------:|
"""
header_md = header_md.strip()

rules = [0, 4, 5, 7]
create_main_table(datasets, methods, metrics, seeds, header, rules, format="tex")
create_main_table(datasets, methods, metrics, seeds, header_md, rules, format="md")


#################################
# Multi-class classification
#################################

results_dir = "../results_online5"
datasets = [
        ("news20_hsm", "News ($m = 20, n = 7532$)"), 
        ("cal101_hsm", "Caltech 101 ($m = 101, n = 4339$)"), 
        ("cal256_hsm", "Caltech 256 ($m = 256, n = 14890$)"), 
        ("aloi.bin_hsm", "Protein ($m = 3548, n = 6621$)"), 
]
metrics = [
    ("micro_f1_k=1", "miF@1", "Micro F1"), 
    ("macro_f1_k=1", "mF@1", "Macro F1"),
    ("macro_f1_k=3", "mF@3", "Macro F1$@3$"),
    ("macro_recall_k=3", "mR@3", "Macro Recall$@3$"), 
    ("macro_precision_k=3", "mP@3", "Macro Prec.$@3$"), 
    ("multi_class_gmean_k=1", "G@1", "G-Mean"),
    ("multi_class_hmean_k=1", "H@1", "H-Mean"),
]
methods = [
    #("default_prediction", "Top-K / $\hat \eta(x) > 0.5$"),
    ("online_default", "Top-$1$ / Top-$k$"),

    ("online_greedy", "Greedy"),
    ("online_frank_wolfe_exp=1.5", "Online-FW"),
    ("omma", "OMMA"),
    #("frank_wolfe_on_test", "FW$(\\boldsymbol{y})$"),

    #("ofo_etu", "OFO$(\\hat \\eta)$"),
    #("online_greedy_etu", "Greedy$(\\hat \\eta)$"),
    ("online_frank_wolfe_etu_exp=1.5", "Online-FW$_{\\hat \\eta}$"),
    ("omma_etu", "OMMA$_{\\hat \\eta}$"),
    #("frank_wolfe_etu", "FW$(\\hat \\eta)$"),
]

# pre_header = "\\begin{tabular}{l|c|cccc|cc|c|cccc|cc}"
# header = "\\toprule\n    Method & Micro & \\multicolumn{4}{c|}{Macro} & & & Micro & \\multicolumn{4}{c|}{Macro} & & \\\\"
# rules = [0, 3]
# create_table(datasets, methods, metrics, seeds, pre_header, header, rules)


## Extended results

In [ ]:
results_dir = "../results_online6"
datasets = [
        ("youtube_deepwalk_plt", "\\datasettable{YouTube} ($m = 46, n = 7926$)"), 
        ("eurlex_lexglue_plt", "\\datasettable{Eurlex-LexGlue} ($m = 100, n = 5000$)"), 
        ("mediamill_plt", "\\datasettable{Mediamill} ($m = 101, n = 12914$)"), 
        ("flicker_deepwalk_plt", "\\datasettable{Flickr} ($m = 195, n = 24154$)"), 
        #("rcv1x_plt", "\\datasettable{RCV1X} ($m = 2456, n = 155962$)"), 
        #("amazoncat_plt", "\\datasettable{AmazonCat} ($m = 13330, n = 306784$)"), 
        
        #("eurlex_plt", "Eurlex"), 
        #("eurlex_lightxml", "Eurlex"), 
        #("eurlex_plt", "Eurlex ($m = 3993, n = 3809$)"), 
        #("eurlex_lightxml", "Eurlex ($m = 5600, n = 5000$)"), 
        #("amazoncat_lightxml", "AmazonCat"), 
        #("amazon_lightxml", "Amazon")
        # "wiki500_lightxml", "Wikipedia",
]
metrics = [
    ("micro_f1_k=0", "miF@0", "Micro F1"), 
    ("macro_f1_k=0", "mF@0", "Macro F1"),
    ("macro_f1_k=3", "mF@3", "Macro F1$@3$"),
    #("macro_f1_k=5", f"mF@5", "Macro F1$@5$"),
    ("macro_recall_k=3", "mR@3", "Macro Recall$@3$"), 
    ("macro_precision_k=3", "mP@3", "Macro Prec.$@3$"), 
    #("macro_recall_k=5", f"mR@5", "Macro Recall$@5$"),
    ("macro_gmean_k=0", "mG@0", "Macro G-Mean"),
    ("macro_hmean_k=0", "mH@0", "Macro H-Mean"),
    #("macro_min_tp_tn_k=0", "min_tp_tn@0", "Macro $\\min(\\text{tp},\\text{tn})$"),
    #("macro_precision", f"mP@{k}", "Macro Precision"), 
    #("macro_min_tp_tn", f"min_tp_tn@{k}", "Macro-Min(TP,TN)"),
]

methods = [
    #("default_prediction", "Top-K / $\hat \eta(x) > 0.5$"),
    ("online_default", "Top-$k$ / $\hat \eta \!>\!0.5$"),

    ("ofo", "OFO"),
    ("online_greedy", "Greedy"),
    ("online_frank_wolfe_exp=1.1", "Online-FW"),
    ("omma", "OMMA-v2"),
    ("online_my", "OMMA"),
    #("frank_wolfe", "FW"),
    ("frank_wolfe_on_test", "FW-test"),

    #("ofo_etu", "OFO$(\\hat \\eta)$"),
    #("online_greedy_etu", "Greedy$(\\hat \\eta)$"),
    ("online_frank_wolfe_etu_exp=1.1", "Online-FW$(\\hat \\eta)$"),
    ("omma_etu", "OMMA-v2$(\\hat \\eta)$"),
    ("online_my", "OMMA$(\\hat \\eta)$"),
    ("frank_wolfe_etu", "FW$(\\hat \\eta)$"),
]

seeds = [1, 13, 23, 2024, 7700]
seeds = [1, 13, 23, 2024, 7700]


# Create data
def create_table(datasets, methods, metrics, seeds, pre_header, header, rules):
    table = []
    for i, (dataset, dataset_name) in enumerate(datasets):
        for method, method_name in methods:
            table.append([method_name])
        
        for metric, metric_key, metric_name in metrics:
            table_col = []
            table_col_std = []

            for method, method_name in methods:
                mean, std = get_result(dataset, method, metric, "pred_utility_history", seeds=seeds)
                table_col.append(mean)
                table_col_std.append(std)
            #print(table_col)
                
            table_col = np.array(table_col).round(2)
            table_col_str = [f"{x:.2f} & \scriptsize $\pm$ {x_std:.2f}" if x != 0 else "& $\\times$" for x, x_std in zip(table_col, table_col_std)]

            max_row = np.argsort(table_col)
            max_i = -1
            table_col_str[max_row[max_i]] = wrap_calls(table_col_str[max_row[max_i]], "textbf")
            max_i -= 1
            while table_col[max_row[max_i]] == table_col[max_row[max_i + 1]] and max_i > -len(table_col):
                table_col_str[max_row[max_i]] = wrap_calls(table_col_str[max_row[max_i]], "textbf")
                max_i -= 1

            if max_i > -len(table_col):
                table_col_str[max_row[max_i]] = wrap_calls(table_col_str[max_row[max_i]], "textit")
                max_i -= 1
            while table_col[max_row[max_i]] == table_col[max_row[max_i + 1]] and max_i > -len(table_col):
                table_col_str[max_row[max_i]] = wrap_calls(table_col_str[max_row[max_i]], "textit")
                max_i -= 1
            #print(table_col_str)
            
            m = len(table_col)
            for j, c in enumerate(table_col_str):
                table[-(m - j)].append(c)

            table_col = []
            table_col_std = []
            
            for method, method_name in methods:
                mean, std = get_result(dataset, method, metric, "time", seeds=seeds)
                table_col.append(mean)
                table_col_std.append(std)
            #print(table_col)
            table_col_str = [f"{x:.2f} & \scriptsize $\pm$ {x_std:.2f}" if x != 0 else "& $\\times$" for x, x_std in zip(table_col, table_col_std)]

            # max_row = np.argsort(-table_col)
            # max_i = -1
            # table_col_str[max_row[max_i]] = f"\\textbf{{{table_col_str[max_row[max_i]]}}}"
            # max_i -= 1
            # while table_col[max_row[max_i]] == table_col[max_row[max_i + 1]]:
            #     table_col_str[max_row[max_i]] = f"\\textbf{{{table_col_str[max_row[max_i]]}}}"
            #     max_i -= 1

            # table_col_str[max_row[max_i]] = f"\\textit{{{table_col_str[max_row[max_i]]}}}"
            # max_i -= 1
            # while table_col[max_row[max_i]] == table_col[max_row[max_i + 1]]:
            #     table_col_str[max_row[max_i]] = f"\\textit{{{table_col_str[max_row[max_i]]}}}"
            #     max_i -= 1
            # print(table_col_str)
            
            m = len(table_col)
            for j, c in enumerate(table_col_str):
                table[-(m - j)].append(c)

    print(pre_header)
    header2 = "\\toprule\n    Method & " + " & ".join([f"\\multicolumn{{4}}{{c|}}{{{metric[2]}}}" for metric in metrics[:-1]]) + f"& \\multicolumn{{4}}{{c}}{{{metrics[-1][2]}}}" + " \\\\"
    header3 = "    " + "& \\multicolumn{2}{c}{(\%)} & \\multicolumn{2}{c|}{time (s)} " * len(metrics[:-1]) + "& \\multicolumn{2}{c}{(\%)} & \\multicolumn{2}{c}{time (s)}" + " \\\\"
    #print(header)
    print(header2)
    print(header3)

    d = 0
    for i, table_row in enumerate(table):
        if i % len(methods) == 0:
            header = f"\\midrule\n& \multicolumn{{28}}{{c}}{{{datasets[d][1]}}}  \\\\ \n \\midrule"
            d += 1
            print(header)
        row = "    " + " & ".join(table_row) + " \\\\"
        for r in rules:
            if i % len(methods) == r:
                row += "\n    \\midrule"
        print(row)

    print("\\bottomrule\n\\end{tabular}")


#pre_header = "\\begin{tabular}{l|cc|cc|cc|cc|cc|cc|cc}"
#header = "\\toprule\n    Method & \\multicolumn{2}{c|}{Micro} & \\multicolumn{12}{c}{Macro} \\\\"
pre_header = "\\begin{tabular}{l|r@{}lr@{}l|r@{}lr@{}l|r@{}lr@{}l|r@{}lr@{}l|r@{}lr@{}l|r@{}lr@{}l|r@{}lr@{}l}"
header = "\\toprule\n    Method & \\multicolumn{4}{c|}{Micro} & \\multicolumn{24}{c}{Macro} \\\\"
rules = [0, 7]
create_table(datasets, methods, metrics, seeds, pre_header, header, rules)

In [ ]:
results_dir = "../results_online6"
datasets = [
        ("youtube_deepwalk_plt", "\\datasettable{YouTube} ($m = 46, n = 7926$)"), 
        ("eurlex_lexglue_plt", "\\datasettable{Eurlex-LexGlue} ($m = 100, n = 5000$)"), 
        ("mediamill_plt", "\\datasettable{Mediamill} ($m = 101, n = 12914$)"), 
        ("flicker_deepwalk_plt", "\\datasettable{Flickr} ($m = 195, n = 24154$)"), 
        #("eurlex_plt", "Eurlex"), 
        #("eurlex_lightxml", "Eurlex"), 
        ("rcv1x_plt", "\\datasettable{RCV1X} ($m = 2456, n = 155962$)"), 
        #("eurlex_plt", "Eurlex ($m = 3993, n = 3809$)"), 
        #("eurlex_lightxml", "Eurlex ($m = 5600, n = 5000$)"), 
        ("amazoncat_plt", "\\datasettable{AmazonCat} ($m = 13330, n = 306784$)"), 
        #("amazoncat_lightxml", "AmazonCat"), 
        #("amazon_lightxml", "Amazon")
        # "wiki500_lightxml", "Wikipedia",
]
metrics = [
    ("micro_f1_k=0", "miF@0", "Micro F1"), 
    ("macro_f1_k=0", "mF@0", "Macro F1"),
    ("macro_f1_k=3", "mF@3", "Macro F1$@3$"),
    #("macro_f1_k=5", f"mF@5", "Macro F1$@5$"),
    ("macro_recall_k=3", "mR@3", "Macro Recall$@3$"), 
    ("macro_precision_k=3", "mP@3", "Macro Prec.$@3$"), 
    #("macro_recall_k=5", f"mR@5", "Macro Recall$@5$"),
    ("macro_gmean_k=0", "mG@0", "Macro G-Mean"),
    ("macro_hmean_k=0", "mH@0", "Macro H-Mean"),
    #("macro_min_tp_tn_k=0", "min_tp_tn@0", "Macro $\\min(\\text{tp},\\text{tn})$"),
    #("macro_precision", f"mP@{k}", "Macro Precision"), 
    #("macro_min_tp_tn", f"min_tp_tn@{k}", "Macro-Min(TP,TN)"),
]
methods = [
    #("default_prediction", "Top-K / $\hat \eta(x) > 0.5$"),
    ("online_default", "Top-$k$ / $\hat \eta \!>\!0.5$"),

    ("ofo", "OFO$(\\boldsymbol{y})$"),
    ("online_greedy", "Greedy$(\\boldsymbol{y}$)"),
    ("online_frank_wolfe_exp=1.1", "Online-FW$(\\boldsymbol{y})$"),
    ("omma", "OMMA$(\\boldsymbol{y})$"),
    #("frank_wolfe_on_test", "FW$(\\boldsymbol{y})$"),

    ("ofo_etu", "OFO$(\\hat \\eta)$"),
    ("online_greedy_etu", "Greedy$(\\hat \\eta)$"),
    ("online_frank_wolfe_etu_exp=1.1", "Online-FW$(\\hat \\eta)$"),
    ("omma_etu", "OMMA$(\\hat \\eta)$"),
    #("frank_wolfe_etu", "FW$(\\hat \\eta)$"),
]

methods = [
    #("default_prediction", "Top-K / $\hat \eta(x) > 0.5$"),
    ("online_default", "Top-$k$ / $\hat \eta \!>\!0.5$"),

    ("ofo", "OFO"),
    ("online_greedy", "Greedy"),
    ("online_frank_wolfe_exp=1.1", "Online-FW"),
    ("omma", "OMMA"),
    #("frank_wolfe_on_test", "FW$(\\boldsymbol{y})$"),

    #("ofo_etu", "OFO$_{\\hat \\eta}$"),
    ("online_greedy_etu", "Greedy$(\\hat \\eta)$"),
    ("online_frank_wolfe_etu_exp=1.1", "Online-FW$(\\hat \\eta)$"),
    ("omma_etu", "OMMA$(\\hat \\eta)$"),
    #("frank_wolfe_etu", "FW$(\\hat \\eta)$"),
]

seeds = [1, 13, 23, 2024, 7700]
seeds = [1, 13, 23, 2024, 7700]


def check_cell(c, command):
    if c.strip() == "":
        return ""
    else:
        return f"\\{command}{{{c.strip()}}}"

def wrap_calls(text, command):
    text = text.split("&")
    return "&".join([check_cell(c, command) for c in text])
    


# Create data
def create_table(datasets, methods, metrics, seeds, pre_header, header, rules):
    table = []
    for i, (dataset, dataset_name) in enumerate(datasets):
        for method, method_name in methods:
            table.append([method_name])
        
        for metric, metric_key, metric_name in metrics:
            table_col = []
            table_col_std = []

            for method, method_name in methods:
                mean, std = get_result(dataset, method, metric, "pred_utility_history", seeds=seeds)
                table_col.append(mean)
                table_col_std.append(std)
            #print(table_col)
                
            table_col = np.array(table_col).round(2)
            table_col_str = [f"{x:.2f} & \scriptsize $\pm$ {x_std:.2f}" if x != 0 else "& $\\times$" for x, x_std in zip(table_col, table_col_std)]

            max_row = np.argsort(table_col)
            max_i = -1
            table_col_str[max_row[max_i]] = wrap_calls(table_col_str[max_row[max_i]], "textbf")
            max_i -= 1
            while table_col[max_row[max_i]] == table_col[max_row[max_i + 1]] and max_i > -len(table_col):
                table_col_str[max_row[max_i]] = wrap_calls(table_col_str[max_row[max_i]], "textbf")
                max_i -= 1

            if max_i > -len(table_col):
                table_col_str[max_row[max_i]] = wrap_calls(table_col_str[max_row[max_i]], "textit")
                max_i -= 1
            while table_col[max_row[max_i]] == table_col[max_row[max_i + 1]] and max_i > -len(table_col):
                table_col_str[max_row[max_i]] = wrap_calls(table_col_str[max_row[max_i]], "textit")
                max_i -= 1
            #print(table_col_str)
            
            m = len(table_col)
            for j, c in enumerate(table_col_str):
                table[-(m - j)].append(c)

            table_col = []
            table_col_std = []
            
            for method, method_name in methods:
                mean, std = get_result(dataset, method, metric, "time", seeds=seeds)
                table_col.append(mean)
                table_col_std.append(std)
            #print(table_col)
            table_col_str = [f"{x:.2f} & \scriptsize $\pm$ {x_std:.2f}" if x != 0 else "& $\\times$" for x, x_std in zip(table_col, table_col_std)]

            # max_row = np.argsort(-table_col)
            # max_i = -1
            # table_col_str[max_row[max_i]] = f"\\textbf{{{table_col_str[max_row[max_i]]}}}"
            # max_i -= 1
            # while table_col[max_row[max_i]] == table_col[max_row[max_i + 1]]:
            #     table_col_str[max_row[max_i]] = f"\\textbf{{{table_col_str[max_row[max_i]]}}}"
            #     max_i -= 1

            # table_col_str[max_row[max_i]] = f"\\textit{{{table_col_str[max_row[max_i]]}}}"
            # max_i -= 1
            # while table_col[max_row[max_i]] == table_col[max_row[max_i + 1]]:
            #     table_col_str[max_row[max_i]] = f"\\textit{{{table_col_str[max_row[max_i]]}}}"
            #     max_i -= 1
            # print(table_col_str)
            
            m = len(table_col)
            for j, c in enumerate(table_col_str):
                table[-(m - j)].append(c)

    print(pre_header)
    header2 = "\\toprule\n    Method & " + " & ".join([f"\\multicolumn{{4}}{{c|}}{{{metric[2]}}}" for metric in metrics[:-1]]) + f"& \\multicolumn{{4}}{{c}}{{{metrics[-1][2]}}}" + " \\\\"
    header3 = "    " + "& \\multicolumn{2}{c}{(\%)} & \\multicolumn{2}{c|}{time (s)} " * len(metrics[:-1]) + "& \\multicolumn{2}{c}{(\%)} & \\multicolumn{2}{c}{time (s)}" + " \\\\"
    #print(header)
    print(header2)
    print(header3)

    d = 0
    for i, table_row in enumerate(table):
        if i % len(methods) == 0:
            header = f"\\midrule\n& \multicolumn{{28}}{{c}}{{{datasets[d][1]}}}  \\\\ \n \\midrule"
            d += 1
            print(header)
        row = "    " + " & ".join(table_row) + " \\\\"
        for r in rules:
            if i % len(methods) == r:
                row += "\n    \\midrule"
        print(row)

    print("\\bottomrule\n\\end{tabular}")


#pre_header = "\\begin{tabular}{l|cc|cc|cc|cc|cc|cc|cc}"
#header = "\\toprule\n    Method & \\multicolumn{2}{c|}{Micro} & \\multicolumn{12}{c}{Macro} \\\\"
pre_header = "\\begin{tabular}{l|r@{}lr@{}l|r@{}lr@{}l|r@{}lr@{}l|r@{}lr@{}l|r@{}lr@{}l|r@{}lr@{}l|r@{}lr@{}l}"
header = "\\toprule\n    Method & \\multicolumn{4}{c|}{Micro} & \\multicolumn{24}{c}{Macro} \\\\"
rules = [0, 4]
create_table(datasets, methods, metrics, seeds, pre_header, header, rules)

## Results for the online CPE

In [ ]:
#seeds = [1, 13, 23, 2024, 7700]
seeds = [1, 13, 23]
results_dir = "../results_online9"
datasets = [
        ("youtube_deepwalk_online", "\\datasettable{YouTube} ($m = 46, n = 7926$)"), 
        ("eurlex_lexglue_online", "\\datasettable{Eurlex-LexGlue} ($m = 100, n = 65000$)"), 
        ("mediamill_online", "\\datasettable{Mediamill} ($m = 101, n = 43907$)"), 
        ("flicker_deepwalk_online", "\\datasettable{Flickr} ($m = 195, n = 80513$)"), 
        #("rcv1x_plt", "\\datasettable{RCV1X} ($m = 2456, n = 155962$)"), 
        #("amazoncat_plt", "\\datasettable{AmazonCat} ($m = 13330, n = 306784$)"), 
        
        #("eurlex_plt", "Eurlex"), 
        #("eurlex_lightxml", "Eurlex"), 
        #("eurlex_plt", "Eurlex ($m = 3993, n = 3809$)"), 
        #("eurlex_lightxml", "Eurlex ($m = 5600, n = 5000$)"), 
        #("amazoncat_lightxml", "AmazonCat"), 
        #("amazon_lightxml", "Amazon")
        # "wiki500_lightxml", "Wikipedia",
]
metrics = [
    ("micro_f1_k=0", "miF@0", "Micro F1"), 
    ("macro_f1_k=0", "mF@0", "Macro F1"),
    ("macro_f1_k=3", "mF@3", "Macro F1$@3$"),
    #("macro_f1_k=5", f"mF@5", "Macro F1$@5$"),
    ("macro_recall_k=3", "mR@3", "Macro Recall$@3$"), 
    ("macro_precision_k=3", "mP@3", "Macro Prec.$@3$"), 
    #("macro_recall_k=5", f"mR@5", "Macro Recall$@5$"),
    ("macro_gmean_k=0", "mG@0", "Macro G-Mean"),
    ("macro_hmean_k=0", "mH@0", "Macro H-Mean"),
    #("macro_min_tp_tn_k=0", "min_tp_tn@0", "Macro $\\min(\\text{tp},\\text{tn})$"),
    #("macro_precision", f"mP@{k}", "Macro Precision"), 
    #("macro_min_tp_tn", f"min_tp_tn@{k}", "Macro-Min(TP,TN)"),
]

methods = [
    #("default_prediction", "Top-K / $\hat \eta(x) > 0.5$"),
    ("online_default", "Top-$k$ / $\hat \eta \!>\!0.5$"),

    ("ofo", "OFO"),
    ("online_greedy", "Greedy"),
    ("online_frank_wolfe_exp=1.1", "Online-FW"),
    ("omma", "OMMA"),
    #("online_my", "OMMA"),
    #("frank_wolfe", "FW"),
    #("frank_wolfe_on_test", "FW"),

    #("ofo_etu", "OFO$(\\hat \\eta)$"),
    #("online_greedy_etu", "Greedy$(\\hat \\eta)$"),
    ("online_frank_wolfe_etu_exp=1.1", "Online-FW$(\\hat \\eta)$"),
    ("omma_etu", "OMMA$(\\hat \\eta)$"),
    #("online_my_etu", "OMMA$(\\hat \\eta)$"),
    #("frank_wolfe_etu", "FW$(\\hat \\eta)$"),
]

rules = [0, 4]
create_main_table(datasets, methods, metrics, seeds, header_md, rules, format="md")
